# Unit 5 Building Full Preprocessing Pipeline for the Titanic Dataset

## Drop Unwanted Titanic Columns

## Handle Missing Values in Titanic Dataset

## Encode Categorical Features and Concatenate

## Handle Missing Values and Feature Scaling